# Phase 1: Training Orchestration (Google Colab & Kaggle)

This notebook orchestrates all training activities for **Google Colab or Kaggle execution** with GPU compute support.

## Important

- This notebook **executes training in Google Colab or Kaggle** (not on Azure ML)
- All computation happens on the platform's GPU
- **Storage & Persistence**:
  - **Google Colab**: Checkpoints are automatically saved to Google Drive for persistence across sessions
  - **Kaggle**: Outputs in `/kaggle/working/` are automatically persisted - no manual backup needed
- The notebook must be **re-runnable end-to-end**
- Uses the dataset path specified in the data config (from `config/data/*.yaml`), typically pointing to a local folder included in the repository
- **Session Management**:
  - **Colab**: Sessions timeout after 12-24 hours (depending on Colab plan). Checkpoints are saved to Drive automatically.
  - **Kaggle**: Sessions have time limits based on your plan. All outputs are automatically saved.


## Step 1: Environment Detection

The notebook automatically detects the execution environment (local, Google Colab, or Kaggle) and adapts its behavior accordingly.


In [134]:
import os
from pathlib import Path
# Detect execution environment
IN_COLAB = "COLAB_GPU" in os.environ or "COLAB_TPU" in os.environ
IN_KAGGLE = "KAGGLE_KERNEL_RUN_TYPE" in os.environ
IS_LOCAL = not IN_COLAB and not IN_KAGGLE
# Set platform-specific constants
if IN_COLAB:
    PLATFORM = "colab"
    BASE_DIR = Path("/content")
    BACKUP_ENABLED = True
elif IN_KAGGLE:
    PLATFORM = "kaggle"
    BASE_DIR = Path("/kaggle/working")
    BACKUP_ENABLED = False
else:
    PLATFORM = "local"
    BASE_DIR = None
    BACKUP_ENABLED = False
print(f"✓ Detected environment: {PLATFORM.upper()}")
print(f"Platform: {PLATFORM}")
print(
    f"Base directory: {BASE_DIR if BASE_DIR else 'Current working directory'}")
print(f"Backup enabled: {BACKUP_ENABLED}")


✓ Detected environment: LOCAL
Platform: local
Base directory: Current working directory
Backup enabled: False


## Step 2: Repository Setup

**Note**: Repository setup is only needed for Colab/Kaggle environments. Local environments should already have the repository cloned.

### For Colab/Kaggle: Clone from Git or Upload Files

Choose one of the following options:

**Option A: Clone from Git (Recommended)**

If your repository is on GitHub/GitLab, clone it:

**For Google Colab:**
```python
!git clone -b gg_final_training_2 https://github.com/longdang193/resume-ner-azureml.git /content/resume-ner-azureml
```

**For Kaggle:**
```python
!git clone -b gg_final_training_2 https://github.com/longdang193/resume-ner-azureml.git /kaggle/working/resume-ner-azureml
```

**Option B: Upload Files**

**For Google Colab:**
1. Use the Colab file browser (folder icon on left sidebar)
2. Upload your project files to `/content/resume-ner-azureml/`
3. Ensure the directory structure matches: `src/`, `config/`, `notebooks/`, etc.

**For Kaggle:**
1. Use the Kaggle file browser (Data tab)
2. Upload your project files to `/kaggle/working/resume-ner-azureml/`
3. Ensure the directory structure matches: `src/`, `config/`, `notebooks/`, etc.

### For Local: Repository Already Exists

Local environments should have the repository already cloned. The notebook will automatically detect the repository location.


In [135]:
# Repository setup - only needed for Colab/Kaggle
if not IS_LOCAL:
    if IN_KAGGLE:
        # For Kaggle
        !git clone -b gg_final_training_2 https://github.com/longdang193/resume-ner-azureml.git /kaggle/working/resume-ner-azureml
    elif IN_COLAB:
        # For Google Colab
        !git clone -b gg_final_training_2 https://github.com/longdang193/resume-ner-azureml.git /content/resume-ner-azureml
else:
    print("✓ Local environment detected - assuming repository already exists")


✓ Local environment detected - assuming repository already exists


### Verify Repository Setup

Verify the repository structure exists:


In [136]:
import sys
from pathlib import Path

# Unified path setup for all environments
if IS_LOCAL:
    # Local: assume notebook is in notebooks/ directory
    NOTEBOOK_DIR = Path.cwd()
    ROOT_DIR = NOTEBOOK_DIR.parent
else:
    # Colab/Kaggle: use fixed paths
    ROOT_DIR = BASE_DIR / "resume-ner-azureml"

SRC_DIR = ROOT_DIR / "src"
CONFIG_DIR = ROOT_DIR / "config"
NOTEBOOK_DIR = ROOT_DIR / "notebooks"

# Verify repository structure
if not ROOT_DIR.exists():
    if IS_LOCAL:
        raise FileNotFoundError(
            f"Repository not found at {ROOT_DIR}\n"
            f"Please ensure you're running this notebook from the notebooks/ directory of the repository."
        )
    else:
        raise FileNotFoundError(
            f"Repository not found at {ROOT_DIR}\n"
            f"Please run Step 2 to clone or upload the repository."
        )

required_dirs = ["src", "config", "notebooks"]
missing_dirs = [d for d in required_dirs if not (ROOT_DIR / d).exists()]

if missing_dirs:
    raise FileNotFoundError(
        f"Missing required directories: {missing_dirs}\n"
        f"Please ensure the repository structure is correct."
    )

# Add to Python path
sys.path.insert(0, str(ROOT_DIR))
sys.path.insert(0, str(SRC_DIR))
import importlib.util
# Import directly from module file to avoid triggering orchestration.__init__.py
# which imports mlflow (not yet installed at this point)
paths_validation_spec = importlib.util.spec_from_file_location(
    "paths_validation",
    SRC_DIR / "infrastructure" / "paths" / "validation.py"
)
paths_validation = importlib.util.module_from_spec(paths_validation_spec)
paths_validation_spec.loader.exec_module(paths_validation)
validate_path_before_mkdir = paths_validation.validate_path_before_mkdir

print(f"✓ Repository found at: {ROOT_DIR}")
print(f"✓ Required directories found: {required_dirs}")
print(f"Notebook directory: {NOTEBOOK_DIR}")
print(f"Project root: {ROOT_DIR}")
print(f"Source directory: {SRC_DIR}")
print(f"Config directory: {CONFIG_DIR}")




✓ Repository found at: /workspaces/resume-ner-azureml
✓ Required directories found: ['src', 'config', 'notebooks']
Notebook directory: /workspaces/resume-ner-azureml/notebooks
Project root: /workspaces/resume-ner-azureml
Source directory: /workspaces/resume-ner-azureml/src
Config directory: /workspaces/resume-ner-azureml/config


## Step 3: Install Dependencies

**For Local**: Use conda environment (instructions below).  
**For Colab/Kaggle**: Install packages via pip (automated below).

### Local Environment Setup

For local execution, create and activate a conda environment:

1. Open a terminal in the project root
2. Create the conda environment: `conda env create -f config/environment/conda.yaml`
3. Activate: `conda activate resume-ner-training`
4. Restart the kernel after activation

### Colab/Kaggle: Automated Installation

PyTorch is usually pre-installed in Colab/Kaggle, but we'll verify and install other required packages.


In [137]:
import torch

# Check PyTorch version and GPU availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"Visible GPUs: {device_count}")
    for i in range(device_count):
        print(f"  GPU {i}: {torch.cuda.get_device_name(i)}")

# Verify PyTorch version meets requirements (>=2.6.0)
torch_version = tuple(map(int, torch.__version__.split('.')[:2]))
if torch_version < (2, 6):
    print(f"⚠ Warning: PyTorch {torch.__version__} may not meet requirements (>=2.6.0)")
    if not IS_LOCAL:
        print("Consider upgrading: !pip install torch>=2.6.0 --upgrade")
else:
    print("✓ PyTorch version meets requirements")


PyTorch version: 2.9.1
CUDA available: False
✓ PyTorch version meets requirements


In [138]:
# Install required packages
if IS_LOCAL:
    print("For local environment, please:")
    print("1. Create conda environment: conda env create -f config/environment/conda.yaml")
    print("2. Activate: conda activate resume-ner-training")
    print("3. Restart kernel after activation")
    print("\nIf you've already done this, you can continue to the next cell.")
    print("\nInstalling Azure ML SDK (required for imports)...")
    # Install Azure ML packages even for local (in case conda env not activated)
    %pip install "azure-ai-ml>=1.0.0" --quiet
    %pip install "azure-identity>=1.12.0" --quiet
    %pip install azureml-defaults --quiet
    %pip install azureml-mlflow --quiet
else:
    # Core ML libraries
    %pip install "transformers>=4.35.0,<5.0.0" --quiet
    %pip install "safetensors>=0.4.0" --quiet
    %pip install "datasets>=2.12.0" --quiet

    # ML utilities
    %pip install "numpy>=1.24.0,<2.0.0" --quiet
    %pip install "pandas>=2.0.0" --quiet
    %pip install "scikit-learn>=1.3.0" --quiet

    # Utilities
    %pip install "pyyaml>=6.0" --quiet
    %pip install "tqdm>=4.65.0" --quiet
    %pip install "seqeval>=1.2.2" --quiet
    %pip install "sentencepiece>=0.1.99" --quiet

    # Experiment tracking
    %pip install mlflow --quiet
    %pip install optuna --quiet

    # Azure ML SDK (required for orchestration imports)
    %pip install "azure-ai-ml>=1.0.0" --quiet
    %pip install "azure-identity>=1.12.0" --quiet
    %pip install azureml-defaults --quiet
    %pip install azureml-mlflow --quiet

    # ONNX support
    %pip install onnxruntime --quiet
    %pip install "onnx>=1.16.0" --quiet
    %pip install "onnxscript>=0.1.0" --quiet

    print("✓ All dependencies installed")

For local environment, please:
1. Create conda environment: conda env create -f config/environment/conda.yaml
2. Activate: conda activate resume-ner-training
3. Restart kernel after activation

If you've already done this, you can continue to the next cell.

Installing Azure ML SDK (required for imports)...
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Step 4: Setup Paths and Import Paths

Python paths are already configured in Step 2. This section verifies the setup.


In [139]:
# Environment detection and platform configuration
# Note: This cell is a duplicate of Cell 2. If Cell 2 was already executed, these variables are already set.
# This cell ensures they're set even if Cell 2 was skipped.
import os
from pathlib import Path

# Detect execution environment
IN_COLAB = "COLAB_GPU" in os.environ or "COLAB_TPU" in os.environ
IN_KAGGLE = "KAGGLE_KERNEL_RUN_TYPE" in os.environ
IS_LOCAL = not IN_COLAB and not IN_KAGGLE

# Set platform-specific constants (only if not already set)
if 'PLATFORM' not in globals():
    if IN_COLAB:
        PLATFORM = "colab"
        BASE_DIR = Path("/content")
        BACKUP_ENABLED = True
        print("✓ Detected: Google Colab environment")
    elif IN_KAGGLE:
        PLATFORM = "kaggle"
        BASE_DIR = Path("/kaggle/working")
        BACKUP_ENABLED = False  # Kaggle outputs are automatically persisted
        print("✓ Detected: Kaggle environment")
    else:
        PLATFORM = "local"
        BASE_DIR = None  # Will use Path.cwd() instead
        BACKUP_ENABLED = False
        print("✓ Detected: Local environment")

if 'PLATFORM' in globals():
    print(f"Platform: {PLATFORM}")
    if BASE_DIR:
        print(f"Base directory: {BASE_DIR}")
    else:
        print(f"Base directory: Will use current working directory")
    print(f"Backup enabled: {BACKUP_ENABLED}")


Platform: local
Base directory: Will use current working directory
Backup enabled: False


In [140]:
import os
import sys
from pathlib import Path

# Setup paths (ROOT_DIR should be set in Cell 2)
# If not, set it here
if 'ROOT_DIR' not in globals():
    if IN_COLAB:
        ROOT_DIR = Path("/content/resume-ner-azureml")
    elif IN_KAGGLE:
        ROOT_DIR = Path("/kaggle/working/resume-ner-azureml")
    else:
        ROOT_DIR = Path("/content/resume-ner-azureml")  # Default to Colab path

SRC_DIR = ROOT_DIR / "src"
CONFIG_DIR = ROOT_DIR / "config"
NOTEBOOK_DIR = ROOT_DIR / "notebooks"

# Add to Python path
sys.path.insert(0, str(ROOT_DIR))
sys.path.insert(0, str(SRC_DIR))

print("Notebook directory:", NOTEBOOK_DIR)
print("Project root:", ROOT_DIR)
print("Source directory:", SRC_DIR)
print("Config directory:", CONFIG_DIR)
print("Platform:", PLATFORM if 'PLATFORM' in globals() else "unknown")
print("In Colab:", IN_COLAB if 'IN_COLAB' in globals() else False)
print("In Kaggle:", IN_KAGGLE if 'IN_KAGGLE' in globals() else False)


Notebook directory: /workspaces/resume-ner-azureml/notebooks
Project root: /workspaces/resume-ner-azureml
Source directory: /workspaces/resume-ner-azureml/src
Config directory: /workspaces/resume-ner-azureml/config
Platform: local
In Colab: False
In Kaggle: False


## Step 5: Mount Google Drive

Mount Google Drive to enable checkpoint persistence across Colab sessions. Checkpoints will be automatically saved to Drive after training completes.


In [141]:
# Google Drive backup/restore functionality
# Uses the DriveBackupStore from orchestration.drive_backup module
# The drive_store is created in Cell 15 (after mounting)

# Backward-compatible wrapper functions (delegate to drive_store)
# These maintain the old API for gradual migration
from pathlib import Path

# Note: drive_store is created in Cell 15 (Mount Google Drive)
# If drive_store is None, backup/restore operations are disabled

def backup_to_drive(source_path: Path, is_directory: bool = False) -> bool:
    """
    Backward-compatible wrapper for drive_store.backup().
    
    Note: Prefer using drive_store.backup() directly for better error handling.
    """
    if not BACKUP_ENABLED or drive_store is None:
        return False
    
    if not source_path.exists():
        print(f"⚠ Warning: Source path does not exist: {source_path}")
        return False
    
    # Map is_directory to expect parameter
    expect = "dir" if is_directory else "file"
    result = drive_store.backup(source_path, expect=expect)
    
    if result.ok:
        print(result)
    else:
        print(f"⚠ Warning: Backup failed: {result.reason}")
    
    return result.ok

def restore_from_drive(local_path: Path, is_directory: bool = False) -> bool:
    """
    Backward-compatible wrapper for drive_store.restore().
    
    Note: Prefer using drive_store.restore() directly for better error handling.
    """
    if not BACKUP_ENABLED or drive_store is None:
        return False
    
    # Map is_directory to expect parameter
    expect = "dir" if is_directory else "file"
    result = drive_store.restore(local_path, expect=expect)
    
    if result.ok:
        print(result)
    else:
        print(f"⚠ Warning: Restore failed: {result.reason}")
    
    return result.ok

def ensure_restored_from_drive(local_path: Path, is_directory: bool = False) -> bool:
    """
    Ensure file/directory exists locally, restoring from Drive if missing.
    
    This is the primary entry point for most use cases.
    """
    if not BACKUP_ENABLED or drive_store is None:
        return False
    
    # Map is_directory to expect parameter
    expect = "dir" if is_directory else "file"
    result = drive_store.ensure_local(local_path)
    
    if result.ok and result.action.value == "copied":
        print(result)
    
    return result.ok

print("✓ Backup/restore wrapper functions defined (using DriveBackupStore)")


✓ Backup/restore wrapper functions defined (using DriveBackupStore)


In [142]:
from pathlib import Path
# Fix numpy/pandas compatibility before importing orchestration modules
try:
    from infrastructure.storage.drive import create_colab_store
except (ValueError, ImportError) as e:
    if "numpy.dtype size changed" in str(e) or "numpy" in str(e).lower():
        print("⚠ Numpy/pandas compatibility issue detected. Fixing...")
        import subprocess
        import sys
        subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "--force-reinstall", "--no-cache-dir", "numpy>=1.24.0,<2.0.0", "pandas>=2.0.0", "--quiet"])
        print("✓ Numpy/pandas reinstalled. Please restart the kernel and re-run this cell.")
        raise RuntimeError("Please restart kernel after numpy/pandas fix")
    else:
        raise

# Mount Google Drive and create backup store (Colab only - Kaggle doesn't need this)
# Uses centralized config from config/paths.yaml
DRIVE_BACKUP_DIR = None
drive_store = None

if IN_COLAB:
    drive_store = create_colab_store(ROOT_DIR, CONFIG_DIR)
    if drive_store:
        BACKUP_ENABLED = True
        DRIVE_BACKUP_DIR = drive_store.backup_root
        print(f"✓ Google Drive mounted")
        print(f"✓ Backup base directory: {DRIVE_BACKUP_DIR}")
        print(f"\nNote: All outputs/ will be mirrored to: {DRIVE_BACKUP_DIR / 'outputs'}")
    else:
        BACKUP_ENABLED = False
        print("⚠ Warning: Could not mount Google Drive. Backup to Google Drive will be disabled.")
elif IN_KAGGLE:
    print("✓ Kaggle environment detected - outputs are automatically persisted (no Drive mount needed)")
    BACKUP_ENABLED = False
else:
    print("⚠ Warning: Unknown environment. Backup to Google Drive will be disabled.")
    BACKUP_ENABLED = False


⚠ Warning: Unknown environment. Backup to Google Drive will be disabled.


## Step P1-3.1: Load Centralized Configs

Load and validate all configuration files. Configs are immutable and will be logged with each job for reproducibility.

**Note**: 
- **Local**: Config files should already exist in the repository
- **Colab/Kaggle**: Config files will be auto-created if missing (useful for fresh environments)


In [143]:
# Optional: Update repository from git (only for Colab/Kaggle if needed)
# Uncomment and run if you need to pull latest changes
# if not IS_LOCAL:
#     !cd {ROOT_DIR} && git fetch origin gg_final_training_2
#     !cd {ROOT_DIR} && git reset --hard origin/gg_final_training_2

In [144]:
# Write/override config files (useful for Colab/Kaggle where file editing is limited)
# Local environments should have configs already in the repo
if IS_LOCAL:
    print("✓ Local environment - assuming config files already exist in repository")
else:
    # Create the experiment config directory if it doesn't exist
    experiment_config_dir = CONFIG_DIR / "experiment"
    experiment_config_dir = validate_path_before_mkdir(experiment_config_dir, context="directory")
    experiment_config_dir.mkdir(parents=True, exist_ok=True)

    config_path = experiment_config_dir / "resume_ner_baseline.yaml"

    # Always write/override the config file (useful for Kaggle where editing is difficult)
    config_content = """
experiment_name: "resume_ner_baseline"

# Relative to the top-level config directory
data_config: "data/resume_tiny.yaml"
model_config: "model/distilbert.yaml"
train_config: "train.yaml"
hpo_config: "hpo/prod.yaml"      # default HPO config; stages can override if needed
env_config: "env/azure.yaml"
benchmark_config: "benchmark.yaml"

# High-level orchestration design:
# - Stages: smoke → hpo → training
# - Smoke and HPO stage backbones are controlled by the HPO config file (search_space.backbone.values)
# - Training stage can target specific backbones via stage config
# - AML experiment names are per-stage, optionally per-backbone

stages:
  smoke:
    # AML experiment base name for smoke tests
    aml_experiment: "resume-ner-smoke"
    # HPO config for smoke/dry run tests (uses smoke.yaml with reduced trials)
    hpo_config: "hpo/smoke.yaml"
    # Backbones are controlled by the HPO config file (hpo_config) via search_space.backbone.values

  hpo:
    # AML experiment base name for HPO sweeps
    aml_experiment: "resume-ner-hpo"
    # HPO config override for production HPO sweep (uses prod.yaml instead of default smoke.yaml)
    hpo_config: "hpo/smoke.yaml"
    # Backbones are controlled by the HPO config file (hpo_config) via search_space.backbone.values

  training:
    # AML experiment base name for final single-run training
    aml_experiment: "resume-ner-train"
    # Final production backbone(s); typically one chosen after HPO
    backbones:
      - "distilbert"

# Optional naming policy for how AML experiments are derived per backbone.
# If true, the orchestrator should build experiment_name as:
#   "<aml_experiment>-<backbone>"
# otherwise it should use "<aml_experiment>" directly and rely on tags
# (stage/backbone) for grouping in AML.
naming:
  include_backbone_in_experiment: true
"""

    config_path.write_text(config_content)

    if config_path.exists():
        print(f"✓ Config overridden at: {config_path}")
    else:
        print(f"✓ Config written to: {config_path}")

✓ Local environment - assuming config files already exist in repository


In [145]:
# Write/override HPO config file (useful for Colab/Kaggle where file editing is limited)
# Local environments should have configs already in the repo
if IS_LOCAL:
    print("✓ Local environment - assuming HPO config files already exist in repository")
else:
    # Create the HPO config directory if it doesn't exist
    hpo_config_dir = CONFIG_DIR / "hpo"
    hpo_config_dir = validate_path_before_mkdir(hpo_config_dir, context="directory")
    hpo_config_dir.mkdir(parents=True, exist_ok=True)

    config_path = hpo_config_dir / "smoke.yaml"

    # Always write/override the config file (useful for Kaggle where editing is difficult)
    config_content = """
search_space:
  backbone:
    type: "choice"
    values: ["distilbert"]  # ["distilbert", "distilroberta"]
    # Note: "deberta" excluded from smoke tests due to CUDA/NVRTC issues on Windows
    # DeBERTa requires nvrtc-builtins64_129.dll which may not be available in all environments
  
  learning_rate:
    type: "loguniform"
    min: 1e-5
    max: 5e-5
  
  batch_size:
    type: "choice"
    values: [4]
  
  dropout:
    type: "uniform"
    min: 0.1
    max: 0.3
  
  weight_decay:
    type: "loguniform"
    min: 0.001
    max: 0.1

sampling:
  algorithm: "random"
  max_trials: 1
  timeout_minutes: 20

# Checkpoint configuration for HPO resume support
# Enables saving study state to SQLite database for resuming interrupted runs
checkpoint:
  enabled: true
  study_name: "hpo_{backbone}_smoke_test_3.67"
  storage_path: "{study_name}/study.db"
  auto_resume: true
  # Only save checkpoints for best trials locally (reduces storage from ~30 GB to ~300 MB)
  save_only_best: true

mlflow:
  # Log best trial checkpoint to MLflow after HPO completes
  # Set to false to disable MLflow checkpoint logging entirely
  log_best_checkpoint: true

early_termination:
  policy: "bandit"
  evaluation_interval: 1
  slack_factor: 0.2
  delay_evaluation: 2

objective:
  metric: "macro-f1"
  goal: "maximize"

# Selection strategy configuration for accuracy-speed tradeoff
selection:
  # Accuracy threshold for speed tradeoff (0.015 = 1.5% relative)
  # If two models are within this accuracy difference, prefer faster model
  # Set to null for accuracy-only selection (default behavior)
  accuracy_threshold: 0.015
  
  # Use relative threshold (percentage of best accuracy) vs absolute difference
  # Relative thresholds are more robust across different accuracy ranges
  # Default: true (recommended)
  use_relative_threshold: true
  
  # Minimum relative accuracy gain to justify slower model (optional)
  # If DeBERTa is < 2% better than DistilBERT, prefer DistilBERT
  # Set to null to disable this check
  min_accuracy_gain: 0.02

k_fold:
  enabled: true
  n_splits: 2
  random_seed: 42
  shuffle: true
  stratified: true

# Refit training configuration
# After HPO completes, train the best trial on the full training dataset
# This creates a canonical checkpoint for production use (instead of using arbitrary fold checkpoints)
refit:
  enabled: true  # Default: enabled. Set to false to skip refit training
  # Optional: Add timeout, max_epochs overrides if needed in the future

# Cleanup configuration for interrupted runs
# Controls automatic cleanup/marking of interrupted runs from previous sessions
cleanup:
  # Disable automatic MLflow cleanup (tagging interrupted runs with code.interrupted=true)
  # Default: true (disabled for speed). Set to false to enable automatic cleanup
  disable_auto_cleanup: true
  
  # Disable automatic Optuna marking (marking RUNNING trials as FAILED)
  # Default: false (enabled). Set to true to disable automatic Optuna state cleanup
  disable_auto_optuna_mark: false
"""

    config_path.write_text(config_content)

    if config_path.exists():
        print(f"✓ HPO config overridden at: {config_path}")
    else:
        print(f"✓ HPO config written to: {config_path}")

✓ Local environment - assuming HPO config files already exist in repository


In [146]:
# Write/override training config file (useful for Colab/Kaggle where file editing is limited)
# Local environments should have configs already in the repo
if IS_LOCAL:
    print("✓ Local environment - assuming training config file already exists in repository")
else:
    # Ensure config directory exists
    CONFIG_DIR = validate_path_before_mkdir(CONFIG_DIR, context="directory")
    CONFIG_DIR.mkdir(parents=True, exist_ok=True)

    config_path = CONFIG_DIR / "train.yaml"

    # Always write/override the config file (useful for Kaggle where editing is difficult)
    config_content = """
# Global Training Defaults
# Applied to all training runs

training:
  epochs: 1  # 5
  batch_size: 2  # 12 
  gradient_accumulation_steps: 2
  learning_rate: 2e-5
  weight_decay: 0.01
  warmup_steps: 500
  max_grad_norm: 1.0
  # Data splitting and model-specific settings
  val_split_divisor: 10  # Divide train set by this to create validation split if none exists
  deberta_max_batch_size: 8  # 16  # Maximum batch size for DeBERTa models (memory constraints)
  warmup_steps_divisor: 10  # Divide total steps by this to cap warmup steps
  
  # EDA-based metric selection
  metric: "macro-f1"  # Class imbalance requires macro-f1
  metric_mode: "max"  # Maximize macro-f1
  
  early_stopping:
    enabled: true
    patience: 3
    min_delta: 0.001

logging:
  log_interval: 100
  eval_interval: 500
  save_interval: 1000

# NOTE: Multi-GPU / DDP is optional and currently experimental. When enabled,
# the training code will use this section together with hardware detection to
# decide whether to run single-GPU vs multi-GPU. If no multiple GPUs or DDP
# backend are available, it will safely fall back to single-GPU.
distributed:
  enabled: false         # Set true to enable multi-GPU / DDP
  backend: "nccl"        # Typically 'nccl' for GPUs
  world_size: "auto"     # 'auto' = use all visible GPUs; or set an int
  init_method: "env://"  # Default init method; can be overridden if needed
  timeout_seconds: 1800  # Process group init timeout (in seconds)
"""

    config_path.write_text(config_content)

    if config_path.exists():
        print(f"✓ Training config overridden at: {config_path}")
    else:
        print(f"✓ Training config written to: {config_path}")

✓ Local environment - assuming training config file already exists in repository


### Define Constants

Define constants for file and directory names used throughout the notebook. Benchmark settings come from centralized config, not hard-coded here. These constants work across all environments.


In [147]:
# Import constants from centralized module
from orchestration import (
    STAGE_HPO,
    STAGE_TRAINING,
    METRICS_FILENAME,
    BENCHMARK_FILENAME,
    CHECKPOINT_DIRNAME,
    DEFAULT_RANDOM_SEED,
    DEFAULT_K_FOLDS,
)

    # Note: These imports still work via orchestration facade for backward compatibility
from orchestration.jobs.tracking.mlflow_tracker import (
    MLflowSweepTracker,
    MLflowBenchmarkTracker,
    MLflowTrainingTracker,
    MLflowConversionTracker,
)


### Define Helper Functions

Reusable helper functions following DRY principle for common operations. These functions work across all environments (local, Colab, Kaggle).


In [148]:
# Import helper functions from consolidated modules (DRY principle)
from typing import List, Optional, Any
from orchestration import (
    build_mlflow_experiment_name,
    setup_mlflow_for_stage,
    run_benchmarking,
)
from common.shared import verify_output_file

# Wrapper function for run_benchmarking that uses notebook-specific paths
def run_benchmarking_local(
    checkpoint_dir: Path,
    test_data_path: Path,
    output_path: Path,
    batch_sizes: List[int],
    iterations: int,
    warmup_iterations: int,
    max_length: int = 512,
    device: Optional[str] = None,
    tracker: Optional[Any] = None,
    backbone: Optional[str] = None,
    benchmark_source: str = "final_training",
    study_key_hash: Optional[str] = None,
    trial_key_hash: Optional[str] = None,
) -> bool:
    """
    Run benchmarking on a model checkpoint (local notebook wrapper).
    
    This is a thin wrapper around orchestration.benchmark_utils.run_benchmarking
    that automatically uses the notebook's SRC_DIR and ROOT_DIR.
    
    Args:
        checkpoint_dir: Path to checkpoint directory.
        test_data_path: Path to test data JSON file.
        output_path: Path to output benchmark.json file.
        batch_sizes: List of batch sizes to test.
        iterations: Number of iterations per batch size.
        warmup_iterations: Number of warmup iterations.
        max_length: Maximum sequence length.
        device: Device to use (None = auto-detect).
        tracker: Optional MLflowBenchmarkTracker instance.
        backbone: Optional model backbone name.
        benchmark_source: Source of benchmark ("hpo_trial" or "final_training").
        study_key_hash: Optional study key hash for grouping tags.
        trial_key_hash: Optional trial key hash for grouping tags.
    
    Returns:
        True if successful, False otherwise.
    """
    return run_benchmarking(
        checkpoint_dir=checkpoint_dir,
        test_data_path=test_data_path,
        output_path=output_path,
        batch_sizes=batch_sizes,
        iterations=iterations,
        warmup_iterations=warmup_iterations,
        max_length=max_length,
        device=device,
        tracker=tracker,
        backbone=backbone,
        benchmark_source=benchmark_source,
        project_root=ROOT_DIR,
        study_key_hash=study_key_hash,
        trial_key_hash=trial_key_hash,
    )


In [149]:
from pathlib import Path
from typing import Any, Dict

from common.constants import EXPERIMENT_NAME
from infrastructure.config.loader import (
    ExperimentConfig,
    compute_config_hashes,
    create_config_metadata,
    load_all_configs,
    load_experiment_config,
    snapshot_configs,
    validate_config_immutability,
)

# P1-3.1: Load Centralized Configs (local-only)
# Mirrors the Azure orchestration notebook, but does not create an Azure ML client.

if not CONFIG_DIR.exists():
    raise FileNotFoundError(f"Config directory not found: {CONFIG_DIR}")

experiment_config: ExperimentConfig = load_experiment_config(CONFIG_DIR, EXPERIMENT_NAME)
configs: Dict[str, Any] = load_all_configs(experiment_config)
config_hashes = compute_config_hashes(configs)
config_metadata = create_config_metadata(configs, config_hashes)

# Immutable snapshots for runtime mutation checks
original_configs = snapshot_configs(configs)
validate_config_immutability(configs, original_configs)

print(f"Loaded experiment: {experiment_config.name}")
print("Loaded config domains:", sorted(configs.keys()))
print("Config hashes:", config_hashes)
print("Config metadata:", config_metadata)

# Get dataset path from data config (centralized configuration)
# The local_path in the data config is relative to the config directory
data_config = configs["data"]
local_path_str = data_config.get("local_path", "../dataset")
DATASET_LOCAL_PATH = (CONFIG_DIR / local_path_str).resolve()

# Check if seed-based dataset structure (for dataset_tiny with seed subdirectories)
seed = data_config.get("seed")
if seed is not None and "dataset_tiny" in str(DATASET_LOCAL_PATH):
    DATASET_LOCAL_PATH = DATASET_LOCAL_PATH / f"seed{seed}"

print(f"Dataset path (from data config): {DATASET_LOCAL_PATH}")
if seed is not None:
    print(f"Using seed: {seed}")


Loaded experiment: resume_ner_baseline
Loaded config domains: ['benchmark', 'data', 'env', 'hpo', 'model', 'train']
Config hashes: {'data': 'e87b126b961fa20d', 'model': '5f90a66353401b44', 'train': '129186d04c5b57c6', 'hpo': '1c47c1143764f7b2', 'env': '3e54b931c7640cf2', 'benchmark': '9c427b69c6e9db79'}
Config metadata: {'data_config_hash': 'e87b126b961fa20d', 'model_config_hash': '5f90a66353401b44', 'train_config_hash': '129186d04c5b57c6', 'hpo_config_hash': '1c47c1143764f7b2', 'env_config_hash': '3e54b931c7640cf2', 'data_version': 'v3', 'model_backbone': 'distilbert-base-uncased'}
Dataset path (from data config): /workspaces/resume-ner-azureml/dataset_tiny/seed0
Using seed: 0


## Step P1-3.2: Verify Local Dataset

Verify that the dataset directory (specified by `local_path` in the data config) exists and contains the required files. The dataset path is loaded from the centralized data configuration in Step P1-3.1.


In [150]:
# P1-3.2: Verify Local Dataset
# The dataset path comes from the data config's local_path field (loaded in Step P1-3.1).
# This ensures the dataset location is controlled by centralized configuration.
# Note: train.json is required, but validation.json is optional (matches training script behavior).

REQUIRED_FILE = "train.json"
OPTIONAL_FILE = "validation.json"

if not DATASET_LOCAL_PATH.exists():
    raise FileNotFoundError(
        f"Dataset directory not found: {DATASET_LOCAL_PATH}\n"
        f"This path comes from the data config's 'local_path' field.\n"
        f"If you need to create the dataset, run the notebook: notebooks/00_make_tiny_dataset.ipynb"
    )

# Check required file
train_file = DATASET_LOCAL_PATH / REQUIRED_FILE
if not train_file.exists():
    raise FileNotFoundError(
        f"Required dataset file not found: {train_file}\n"
        f"This path comes from the data config's 'local_path' field.\n"
        f"If you need to create it, run the notebook: notebooks/00_make_tiny_dataset.ipynb"
    )

# Check optional file
val_file = DATASET_LOCAL_PATH / OPTIONAL_FILE
has_validation = val_file.exists()

print(f"✓ Dataset directory found: {DATASET_LOCAL_PATH}")
print(f"  (from data config: {data_config.get('name', 'unknown')} v{data_config.get('version', 'unknown')})")

train_size = train_file.stat().st_size
print(f"  ✓ {REQUIRED_FILE} ({train_size:,} bytes)")

if has_validation:
    val_size = val_file.stat().st_size
    print(f"  ✓ {OPTIONAL_FILE} ({val_size:,} bytes)")
else:
    print(f"  ⚠ {OPTIONAL_FILE} not found (optional - training will proceed without validation set)")


✓ Dataset directory found: /workspaces/resume-ner-azureml/dataset_tiny/seed0
  (from data config: resume-ner-data-tiny-short vv3)
  ✓ train.json (27,315 bytes)
  ⚠ validation.json not found (optional - training will proceed without validation set)


## Step P1-3.2.1: Optional Train/Test Split

**Optional step**: Create a train/test split if `test.json` is missing. This is useful when you only have `train.json` and `validation.json` and want to create a separate test set.

**⚠ WARNING**: This will overwrite `train.json` with the split version. Only enable if you want to create a permanent train/test split.


In [151]:
# Optional: create train/test split if test.json is missing
# WARNING: This will overwrite train.json with the split version
# Only enable if you want to create a permanent train/test split
import json
from pathlib import Path
from typing import List, Dict, Any, Tuple, Optional

from data.loaders.dataset_loader import split_train_test, save_split_files

CREATE_TEST_SPLIT = False  # Set True to create test.json when absent (WARNING: overwrites train.json)

train_file = DATASET_LOCAL_PATH / "train.json"
val_file = DATASET_LOCAL_PATH / "validation.json"
test_file = DATASET_LOCAL_PATH / "test.json"

if CREATE_TEST_SPLIT and not test_file.exists():
    # Backup original train.json before overwriting
    backup_file = DATASET_LOCAL_PATH / "train.json.backup"
    if train_file.exists() and not backup_file.exists():
        import shutil
        shutil.copy2(train_file, backup_file)
        print(f"⚠ Backed up original train.json to {backup_file}")
    
    full_dataset = []
    # Start with train data; optionally include validation to maximize coverage
    with open(train_file, "r", encoding="utf-8") as f:
        full_dataset.extend(json.load(f))
    if val_file.exists():
        with open(val_file, "r", encoding="utf-8") as f:
            full_dataset.extend(json.load(f))

    split_cfg = configs.get("data", {}).get("splitting", {})
    train_ratio = split_cfg.get("train_test_ratio", 0.8)
    stratified = split_cfg.get("stratified", False)
    random_seed = split_cfg.get("random_seed", 42)
    entity_types = configs.get("data", {}).get("schema", {}).get("entity_types", [])

    print(f"Creating train/test split (train_ratio={train_ratio}, stratified={stratified})...")
    print(f"⚠ WARNING: This will overwrite train.json with {int(len(full_dataset) * train_ratio)} samples")
    
    new_train, new_test = split_train_test(
        dataset=full_dataset,
        train_ratio=train_ratio,
        stratified=stratified,
        random_seed=random_seed,
        entity_types=entity_types,
    )

    save_split_files(DATASET_LOCAL_PATH, new_train, new_test)
    print(f"✓ Wrote train.json ({len(new_train)}) and test.json ({len(new_test)})")
elif test_file.exists():
    print(f"✓ Found existing test.json at {test_file}")
else:
    print("⚠ test.json not found. Set CREATE_TEST_SPLIT=True to generate a split.")


✓ Found existing test.json at /workspaces/resume-ner-azureml/dataset_tiny/seed0/test.json


## Step P1-3.3: Setup Local Environment

Verify GPU availability, set up MLflow tracking (local file store), and check that key dependencies are installed. This step ensures the local environment is ready for training.


In [152]:
import sys
import torch

DEFAULT_DEVICE = "cuda"

env_config = configs["env"]
device_type = env_config.get("compute", {}).get("device", DEFAULT_DEVICE)

# Fallback to CPU if CUDA is requested but not available
if device_type == "cuda" and not torch.cuda.is_available():
    print("⚠ Warning: CUDA device requested but not available. Falling back to CPU.")
    if not IS_LOCAL:
        print("  In Colab, ensure you've selected a GPU runtime: Runtime > Change runtime type > GPU")
    device_type = "cpu"


⚠ Warning: CUDA device requested but not available. Falling back to CPU.


In [153]:
from pathlib import Path
import mlflow
from common.shared.mlflow_setup import setup_mlflow_from_config

# Get MLflow tracking URI for later use
mlflow_tracking_uri = mlflow.get_tracking_uri()
if mlflow_tracking_uri:
    print(f"MLflow tracking URI: {mlflow_tracking_uri[:80]}...")
else:
    print("Warning: MLflow tracking URI not set")

# Setup MLflow from config (automatically uses Azure ML if enabled in config/mlflow.yaml)
# To enable Azure ML Workspace tracking:
# 1. Edit config/mlflow.yaml and set azure_ml.enabled: true
# 2. Set environment variables: AZURE_SUBSCRIPTION_ID and AZURE_RESOURCE_GROUP
setup_mlflow_from_config(
    experiment_name="placeholder",  # Will be set per HPO run
    config_dir=CONFIG_DIR
)

MLflow tracking URI: azureml://germanywestcentral.api.azureml.ms/mlflow/v2.0/subscriptions/50c06ef8-6...


2026-01-11 22:58:56,305 - common.shared.mlflow_setup - INFO - Azure ML enabled in config, attempting to connect...
2026-01-11 22:58:56,308 - common.shared.mlflow_setup - WARNING - [DEBUG] Initial env check - subscription_id: True, resource_group: True, client_id: True, client_secret: True, tenant_id: True
2026-01-11 22:58:56,309 - common.shared.mlflow_setup - WARNING - [DEBUG] Platform detected: local
2026-01-11 22:58:56,309 - common.shared.mlflow_setup - WARNING - [DEBUG] Service Principal check - client_id present: True, client_secret present: True, tenant_id present: True, has_service_principal: True
2026-01-11 22:58:56,311 - common.shared.mlflow_setup - INFO - Using Service Principal authentication (from config.env)
2026-01-11 22:58:56,419 - common.shared.mlflow_setup - INFO - Successfully connected to Azure ML workspace: resume-ner-ws
2026-01-11 23:00:57,868 - common.shared.mlflow_setup - INFO - Using Azure ML workspace tracking


'azureml://germanywestcentral.api.azureml.ms/mlflow/v2.0/subscriptions/50c06ef8-627b-46d5-b779-d07c9b398f75/resourceGroups/resume_ner_2026-01-02-16-47-05/providers/Microsoft.MachineLearningServices/workspaces/resume-ner-ws'

In [154]:
# For Kaggle only - install specific package versions required for Optuna checkpointing
if IN_KAGGLE:
    %pip install ""SQLAlchemy<2.0.0" "alembic<1.13.0" "optuna<4.0.0"" --quiet
else:
    print("Skipping Kaggle-specific package installation (not running on Kaggle)")


Skipping Kaggle-specific package installation (not running on Kaggle)


In [155]:
try:
    import mlflow
    import transformers
    import optuna
except ImportError as e:
    raise ImportError(f"Required package not installed: {e}")

REQUIRED_PACKAGES = {
    "torch": torch,
    "transformers": transformers,
    "mlflow": mlflow,
    "optuna": optuna,
}

for name, module in REQUIRED_PACKAGES.items():
    if not hasattr(module, "__version__"):
        raise ImportError(
            f"Required package '{name}' is not properly installed")

## Step P1-3.4: The Sweep (HPO) - Local with Optuna

Run the full hyperparameter optimization sweep using Optuna to systematically search for the best model configuration. Uses the production HPO configuration with more trials than the dry run.

**Note on K-Fold Cross-Validation:**
- When k-fold CV is enabled (`k_fold.enabled: true`), each trial trains **k models** (one per fold) and returns the **average metric** across folds
- The number of **trials** is controlled by `sampling.max_trials` (e.g., 2 trials in smoke.yaml)
- With k=5 folds and 2 trials: **2 trials × 5 folds = 10 model trainings total**
- K-fold CV provides more robust hyperparameter evaluation but increases compute time (k× per trial)

**Note on Checkpoint and Resume:**
- When `checkpoint.enabled: true` is set in the HPO config, the system automatically saves the Optuna study state to a SQLite database
- This allows interrupted HPO runs to be resumed from the last checkpoint
- The checkpoint is automatically detected and loaded on the next run if `auto_resume: true` (default)
- Platform-specific paths are handled automatically (local, Colab, Kaggle)
- **Selective Checkpoint Saving**: When `checkpoint.save_only_best: true` is set, only best trial checkpoints are saved locally (reduces storage from ~30 GB to ~300 MB for 100 trials)
- **MLflow Checkpoint Logging**: When `mlflow.log_best_checkpoint: true` is set, the best trial checkpoint is automatically logged to MLflow after HPO completes (artifact path: `best_trial_checkpoint`)
- **Refit Training**: When `refit.enabled: true` is set (default), after HPO completes, the best trial is automatically retrained on the full training dataset. This produces a canonical checkpoint in `trial_<n>_<ts>/refit/checkpoint/` that is preferred over fold checkpoints for benchmarking and production use.
- See `docs/HPO_CHECKPOINT_RESUME.md` for detailed documentation


In [156]:
from pathlib import Path
from orchestration import STAGE_HPO
from training.hpo import run_local_hpo_sweep

# Use new paths module (orchestration.paths is deprecated)
from infrastructure.paths import resolve_output_path

# Use centralized HPO root from paths.yaml (respects env_overrides / storage_env)
HPO_ROOT = resolve_output_path(ROOT_DIR, CONFIG_DIR, "hpo")

# Keep fold_splits as a study-level meta artifact, not mixed with trials
HPO_META_DIR = validate_path_before_mkdir(HPO_ROOT / "_meta", context="directory")
HPO_META_DIR.mkdir(parents=True, exist_ok=True)


In [157]:
# Use HPO config already loaded in configs (from Step P1-3.1)
# Following DRY principle - don't reload configs that are already available
# Check for stage-specific hpo_config override
from infrastructure.naming.experiments import get_stage_config
from common.shared.yaml_utils import load_yaml

hpo_stage_config = get_stage_config(experiment_config, STAGE_HPO)
hpo_config_override = hpo_stage_config.get("hpo_config")

if hpo_config_override:
    # Load stage-specific HPO config override
    hpo_config_path = CONFIG_DIR / hpo_config_override
    hpo_config = load_yaml(hpo_config_path)
    print(f"✓ Using stage-specific HPO config for hpo: {hpo_config_override}")
else:
    # Use default HPO config from top-level experiment config
    # Always reload default HPO config from file (don't use cached configs["hpo"])
    # This ensures changes to the YAML file are picked up even if configs dict wasn't reloaded
    # This is especially important in Colab where configs might be cached in memory
    # after editing YAML files without restarting the kernel
    hpo_config_path = experiment_config.hpo_config
    hpo_config = load_yaml(hpo_config_path)
    print(f"✓ Using default HPO config (reloaded from file): {experiment_config.hpo_config.name}")
train_config = configs["train"]
backbone_values = hpo_config["search_space"]["backbone"]["values"]


✓ Using stage-specific HPO config for hpo: hpo/smoke.yaml


### Setup K-Fold Splits and Google Drive Backup for HPO Trials

**K-Fold Cross-Validation Setup**: If k-fold CV is enabled in the HPO config, create and save fold splits before starting the sweep.

**Colab-specific feature**: Configure automatic backup of each HPO trial to Google Drive immediately after completion. This prevents data loss if the Colab session disconnects during long-running hyperparameter optimization sweeps.

**Note on Checkpoint Backup:**
- If `checkpoint.save_only_best: true` is enabled, only best trial checkpoints are saved locally and backed up to Drive
- Each trial's `metrics.json` is always saved and backed up
- The best trial checkpoint is also automatically logged to MLflow (if `mlflow.log_best_checkpoint: true`)
- This reduces storage usage while ensuring the best model is always available


In [158]:
from training.cv_utils import (
    create_kfold_splits,
    save_fold_splits,
    validate_splits,
)
from data.loaders import load_dataset
from infrastructure.paths import resolve_output_path

# Setup k-fold splits if enabled
k_fold_config = hpo_config.get("k_fold", {})
k_folds_enabled = k_fold_config.get("enabled", False)
fold_splits_file = None

if k_folds_enabled:
    n_splits = k_fold_config.get("n_splits", DEFAULT_K_FOLDS)
    random_seed = k_fold_config.get("random_seed", DEFAULT_RANDOM_SEED)
    shuffle = k_fold_config.get("shuffle", True)
    stratified = k_fold_config.get("stratified", False)
    entity_types = (
        configs.get("data", {})
        .get("schema", {})
        .get("entity_types", [])
    )

    print(f"Setting up {n_splits}-fold cross-validation splits...")

    full_dataset = load_dataset(str(DATASET_LOCAL_PATH))
    train_data = full_dataset.get("train", [])

    fold_splits = create_kfold_splits(
        dataset=train_data,
        k=n_splits,
        random_seed=random_seed,
        shuffle=shuffle,
        stratified=stratified,
        entity_types=entity_types,
    )

    # Optional validation to ensure rare entities appear across folds
    validate_splits(train_data, fold_splits, entity_types=entity_types)

    # Use centralized HPO root from paths.yaml (respects env_overrides / storage_env)
    HPO_ROOT = resolve_output_path(ROOT_DIR, CONFIG_DIR, "hpo")

    # Keep fold_splits as a study-level meta artifact
    HPO_META_DIR = validate_path_before_mkdir(
        HPO_ROOT / "_meta", context="directory"
    )
    HPO_META_DIR.mkdir(parents=True, exist_ok=True)

    fold_splits_file = HPO_META_DIR / "fold_splits.json"

    save_fold_splits(
        fold_splits,
        fold_splits_file,
        metadata={
            "k": n_splits,
            "random_seed": random_seed,
            "shuffle": shuffle,
            "stratified": stratified,
            "dataset_path": str(DATASET_LOCAL_PATH),
        },
    )

    print(f"✓ K-fold splits saved to: {fold_splits_file}")

else:
    print("K-fold CV disabled - using single train/validation split")

Setting up 2-fold cross-validation splits...
[CV] Fold 0: {'SKILL': 154} | Missing: ['EDUCATION', 'DESIGNATION', 'EXPERIENCE', 'NAME', 'EMAIL', 'PHONE', 'LOCATION']
[CV] Fold 1: {'SKILL': 107, 'LOCATION': 4, 'DESIGNATION': 1, 'EXPERIENCE': 1, 'EDUCATION': 1} | Missing: ['NAME', 'EMAIL', 'PHONE']
✓ K-fold splits saved to: /workspaces/resume-ner-azureml/outputs/hpo/_meta/fold_splits.json


In [159]:
# Checkpoint functionality is now handled automatically by run_local_hpo_sweep
# when checkpoint.enabled: true is set in the HPO config.
# No manual backup callbacks are needed - SQLite persistence is built-in.


In [160]:
# Checkpoint functionality is now handled automatically by run_local_hpo_sweep
# when checkpoint.enabled: true is set in the HPO config.
# No wrapper functions are needed - SQLite persistence is built-in.


In [161]:
# # In a Kaggle notebook cell
# !cd /kaggle/working/resume-ner-azureml && git fetch origin gg_final_training_2 && git checkout origin/gg_final_training_2 -- src/train.py src/training/trainer.py

In [162]:
# Extract checkpoint configuration from HPO config
checkpoint_config = hpo_config.get("checkpoint", {})

hpo_studies = {}
k_folds_param = k_fold_config.get("n_splits", DEFAULT_K_FOLDS) if k_folds_enabled else None

# Use new centralized naming system for HPO
# Build base output directory: outputs/hpo/<env>/<model>/
# Trial-specific paths will be created by run_local_hpo_sweep as subdirectories

# Import required functions
from pathlib import Path
from orchestration import STAGE_HPO
from training.hpo import run_local_hpo_sweep
from infrastructure.naming.experiments import build_mlflow_experiment_name
from infrastructure.paths.validation import validate_path_before_mkdir
from common.shared.platform_detection import detect_platform

# Ensure environment is defined
environment = detect_platform()
print(f"Detected environment: {environment}")

for backbone in backbone_values:
    mlflow_experiment_name = build_mlflow_experiment_name(
        experiment_config.name, STAGE_HPO, backbone
    )
    
    backbone_name = backbone.split("-")[0] if "-" in backbone else backbone
    
    # Build base HPO directory using new structure: outputs/hpo/<env>/<model>/
    backbone_output_dir = ROOT_DIR / "outputs" / "hpo" / environment / backbone_name
    backbone_output_dir = validate_path_before_mkdir(backbone_output_dir, context="directory")
    backbone_output_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"✓ HPO output directory: {backbone_output_dir}")
    
    # Create restore function for HPO checkpoint if checkpointing enabled and BACKUP_ENABLED
    restore_fn = None
    if checkpoint_config.get("enabled", False) and BACKUP_ENABLED:
        # Resolve study_name from checkpoint_config (same logic as create_study_name)
        study_name_template = checkpoint_config.get("study_name") or hpo_config.get("study_name")
        study_name = None
        if study_name_template:
            study_name = study_name_template.replace("{backbone}", backbone)
        
        # Resolve storage_path with both {backbone} and {study_name} placeholders
        storage_path_template = checkpoint_config.get("storage_path", "{backbone}/study.db")
        storage_path_str = storage_path_template.replace("{backbone}", backbone)
        if study_name:
            storage_path_str = storage_path_str.replace("{study_name}", study_name)
        expected_checkpoint = backbone_output_dir / storage_path_str
        
        def make_restore_fn(checkpoint_path):
            def restore_fn_inner(path: Path) -> bool:
                # Only restore if path matches expected checkpoint
                if path == checkpoint_path:
                    return ensure_restored_from_drive(checkpoint_path, is_directory=False)
                return False
            return restore_fn_inner
        
        restore_fn = make_restore_fn(expected_checkpoint)
    
    # Use standard run_local_hpo_sweep with checkpoint_config
    # Checkpoint.enabled handles persistence via SQLite (better than manual Drive backup)
    study = run_local_hpo_sweep(
        dataset_path=str(DATASET_LOCAL_PATH),
        config_dir=CONFIG_DIR,
        backbone=backbone,
        hpo_config=hpo_config,
        train_config=train_config,
        output_dir=backbone_output_dir,
        mlflow_experiment_name=mlflow_experiment_name,
        k_folds=k_folds_param,
        fold_splits_file=fold_splits_file,
        checkpoint_config=checkpoint_config,
        restore_from_drive=restore_fn,
        data_config=configs.get("data"),
        benchmark_config=configs.get("benchmark"),
    )
    # Backup HPO study.db and study folder to Drive
    # Note: HPO backup function may still be in orchestration.jobs.hpo
    from orchestration.jobs.hpo import backup_hpo_study_to_drive
    
    backup_hpo_study_to_drive(
        backbone=backbone,
        backbone_output_dir=backbone_output_dir,
        checkpoint_config=checkpoint_config,
        hpo_config=hpo_config,
        backup_to_drive=backup_to_drive,
        backup_enabled=BACKUP_ENABLED,
    )

    # Store study in hpo_studies dict (must be inside loop!)
    hpo_studies[backbone] = study



2026-01-11 23:00:57,979 - training.hpo.core.study - INFO - [HPO] Starting optimization for distilbert with checkpointing...


Detected environment: local
✓ HPO output directory: /workspaces/resume-ner-azureml/outputs/hpo/local/distilbert


2026-01-11 23:00:58,114 - infrastructure.naming.mlflow.config - INFO - [Auto-Increment Config] Loading from config_dir=/workspaces/resume-ner-azureml/config, raw_auto_inc_config={'enabled': True, 'processes': {'hpo': True, 'benchmarking': True}, 'format': '{base}.{version}'}
2026-01-11 23:00:58,114 - infrastructure.naming.mlflow.config - INFO - [Auto-Increment Config] Validated config: {'enabled': True, 'processes': {'hpo': True, 'benchmarking': True}, 'format': '{base}.{version}'}, process_type=hpo
2026-01-11 23:00:58,115 - orchestration.jobs.tracking.index.version_counter - INFO - [Reserve Version] Starting reservation: counter_key=resume-ner:hpo:7580f7593872b0c166ace1a5f2e713aed06a37f8bd0dd..., root_dir=/workspaces/resume-ner-azureml, config_dir=/workspaces/resume-ner-azureml/config, counter_path=/workspaces/resume-ner-azureml/outputs/cache/run_name_counter.json
2026-01-11 23:00:58,116 - orchestration.jobs.tracking.index.version_counter - INFO - [Reserve Version] Loaded 0 existing a

🏃 View run local_distilbert_hpo_trial_study-584922ce_t00_1 at: https://germanywestcentral.api.azureml.ms/mlflow/v2.0/subscriptions/50c06ef8-627b-46d5-b779-d07c9b398f75/resourceGroups/resume_ner_2026-01-02-16-47-05/providers/Microsoft.MachineLearningServices/workspaces/resume-ner-ws/#/experiments/2ff6dc5c-9384-4db6-ae7e-53869501b3db/runs/b15e56b3-8435-4168-aa01-1009604a8004
🧪 View experiment at: https://germanywestcentral.api.azureml.ms/mlflow/v2.0/subscriptions/50c06ef8-627b-46d5-b779-d07c9b398f75/resourceGroups/resume_ner_2026-01-02-16-47-05/providers/Microsoft.MachineLearningServices/workspaces/resume-ner-ws/#/experiments/2ff6dc5c-9384-4db6-ae7e-53869501b3db


2026-01-11 23:01:41,088 - infrastructure.tracking.mlflow.lifecycle - INFO - Successfully terminated run b15e56b3-843... with status FINISHED
2026-01-11 23:01:42,103 - training.hpo.trial.callback - INFO - 
2026-01-11 23:01:42,104 - training.hpo.trial.callback - INFO - [BEST]: trial_0
2026-01-11 23:01:42,105 - training.hpo.trial.callback - INFO -   Metrics: macro-f1=0.362666
2026-01-11 23:01:42,105 - training.hpo.trial.callback - INFO -   Params: learning_rate=1.77e-05 | batch_size=4 | dropout=0.167828 | weight_decay=0.022548 (Run ID: b15e56b3-843...)
Best trial: 0. Best value: 0.362666:  50%|█████     | 1/2 [00:42<00:42, 42.09s/it, 42.08/1200 seconds]2026-01-11 23:01:42,254 - training.hpo.execution.local.sweep - INFO - [Trial 1] fold_splits=present, using CV path
2026-01-11 23:01:42,254 - training.hpo.execution.local.sweep - INFO - [Trial 1] Running 2-fold CV
2026-01-11 23:01:42,256 - infrastructure.naming.mlflow.config - INFO - [Auto-Increment Config] Loading from config_dir=/workspace

...

🏃 View run local_distilroberta_hpo_trial_study-25cad2a2_t01_1 at: https://germanywestcentral.api.azureml.ms/mlflow/v2.0/subscriptions/50c06ef8-627b-46d5-b779-d07c9b398f75/resourceGroups/resume_ner_2026-01-02-16-47-05/providers/Microsoft.MachineLearningServices/workspaces/resume-ner-ws/#/experiments/b5a68c09-5298-4f47-ba70-8562f26f7015/runs/aa100db0-e110-4b02-8a60-6035599c8f72
🧪 View experiment at: https://germanywestcentral.api.azureml.ms/mlflow/v2.0/subscriptions/50c06ef8-627b-46d5-b779-d07c9b398f75/resourceGroups/resume_ner_2026-01-02-16-47-05/providers/Microsoft.MachineLearningServices/workspaces/resume-ner-ws/#/experiments/b5a68c09-5298-4f47-ba70-8562f26f7015


2026-01-11 23:04:28,441 - infrastructure.tracking.mlflow.lifecycle - INFO - Successfully terminated run aa100db0-e11... with status FINISHED
2026-01-11 23:04:29,874 - training.hpo.trial.callback - INFO - 
2026-01-11 23:04:29,875 - training.hpo.trial.callback - INFO - [Trial 1]: trial_1
2026-01-11 23:04:29,876 - training.hpo.trial.callback - INFO -   Metrics: macro-f1=0.280783
2026-01-11 23:04:29,876 - training.hpo.trial.callback - INFO -   Params: learning_rate=1.85e-05 | batch_size=4 | dropout=0.161244 | weight_decay=0.002599 (Run ID: aa100db0-e11...)
Best trial: 0. Best value: 0.419576: 100%|██████████| 2/2 [01:24<00:00, 42.12s/it, 84.23/1200 seconds]
2026-01-11 23:04:29,882 - infrastructure.tracking.mlflow.trackers.sweep_tracker - INFO - [LOG_FINAL_METRICS] Starting log_final_metrics for parent_run_id=e1cfc27f-aa6...
2026-01-11 23:04:30,014 - infrastructure.tracking.mlflow.trackers.sweep_tracker - INFO - [LOG_FINAL_METRICS] Found 2 completed trials out of 2 total
2026-01-11 23:04:30

🏃 View run local_distilroberta_hpo_refit_study-25cad2a2_trial-96bf728b_t00_1 at: https://germanywestcentral.api.azureml.ms/mlflow/v2.0/subscriptions/50c06ef8-627b-46d5-b779-d07c9b398f75/resourceGroups/resume_ner_2026-01-02-16-47-05/providers/Microsoft.MachineLearningServices/workspaces/resume-ner-ws/#/experiments/b5a68c09-5298-4f47-ba70-8562f26f7015/runs/4452788f-ce38-4b6c-a849-d6d3250539f0
🧪 View experiment at: https://germanywestcentral.api.azureml.ms/mlflow/v2.0/subscriptions/50c06ef8-627b-46d5-b779-d07c9b398f75/resourceGroups/resume_ner_2026-01-02-16-47-05/providers/Microsoft.MachineLearningServices/workspaces/resume-ner-ws/#/experiments/b5a68c09-5298-4f47-ba70-8562f26f7015


2026-01-11 23:05:12,193 - infrastructure.tracking.mlflow.lifecycle - INFO - Successfully terminated run 4452788f-ce3... with status FINISHED
2026-01-11 23:05:12,193 - training.hpo.execution.local.sweep - INFO - [REFIT] ✓ Artifacts uploaded and run marked as FINISHED: 4452788f-ce3...
2026-01-11 23:05:12,194 - training.hpo.checkpoint.cleanup - INFO - Final cleanup: kept checkpoints for best trial 0 (metric=0.419576, CV=no, refit=no), deleted 1 non-best checkpoints
2026-01-11 23:05:12,195 - infrastructure.tracking.mlflow.trackers.sweep_tracker - INFO - [START_SWEEP_RUN] Context manager exiting normally. run_id=e1cfc27f-aa6...


🏃 View run local_distilroberta_hpo_study-25cad2a2_smoke_test_path_testing_23_1 at: https://germanywestcentral.api.azureml.ms/mlflow/v2.0/subscriptions/50c06ef8-627b-46d5-b779-d07c9b398f75/resourceGroups/resume_ner_2026-01-02-16-47-05/providers/Microsoft.MachineLearningServices/workspaces/resume-ner-ws/#/experiments/b5a68c09-5298-4f47-ba70-8562f26f7015/runs/e1cfc27f-aa6c-48f1-b156-7414d694c148
🧪 View experiment at: https://germanywestcentral.api.azureml.ms/mlflow/v2.0/subscriptions/50c06ef8-627b-46d5-b779-d07c9b398f75/resourceGroups/resume_ner_2026-01-02-16-47-05/providers/Microsoft.MachineLearningServices/workspaces/resume-ner-ws/#/experiments/b5a68c09-5298-4f47-ba70-8562f26f7015


In [163]:
# Generate missing trial_meta.json files for existing trials
from training.hpo.trial.meta import generate_missing_trial_meta_for_all_studies

if BACKUP_ENABLED and "hpo_studies" in locals():
    total_created = generate_missing_trial_meta_for_all_studies(
        hpo_studies=hpo_studies if "hpo_studies" in locals() else {},
        backbone_values=backbone_values,
        root_dir=ROOT_DIR,
        environment=environment,
        hpo_config=hpo_config,
        data_config=data_config if "data_config" in locals() else None,
        backup_enabled=BACKUP_ENABLED,
    )
    print(f"\n[OK] Total: Created {total_created} trial_meta.json files")
else:
    print("[INFO] Skipping trial_meta.json generation (BACKUP_ENABLED=False or hpo_studies not available)")



[INFO] Skipping trial_meta.json generation (BACKUP_ENABLED=False or hpo_studies not available)


In [164]:
from evaluation.selection.study_summary import print_study_summaries
from common.shared.platform_detection import detect_platform

# Get environment if not already set
if 'environment' not in locals():
    environment = detect_platform()

# Print study summaries using the module
print_study_summaries(
    hpo_studies=hpo_studies if "hpo_studies" in locals() else None,
    backbone_values=backbone_values if "backbone_values" in locals() else [],
    hpo_config=hpo_config,
    root_dir=ROOT_DIR,
    environment=environment,
)


📊 distilbert: 2 trials, best macro-f1=0.3627 [study-584922ce, trial-c4b9e536, t0]
   CV: 0.3627 ± 0.1354
📊 distilroberta: 2 trials, best macro-f1=0.4196 [study-25cad2a2, trial-96bf728b, t0]
   CV: 0.4196 ± 0.0215


## Step P1-3.5: Benchmarking Best Trials

Benchmark the best trial from each backbone to measure actual inference performance. This provides real latency data that replaces parameter-count proxies in model selection, enabling more accurate speed comparisons.

**Workflow:**
1. Identify best trial per backbone (from HPO results)
2. Select checkpoint: prefers `refit/checkpoint/` (if refit training completed), otherwise uses best fold from `cv/foldN/checkpoint/`
3. Run benchmarking on each best trial checkpoint
4. Save benchmark results as `benchmark.json` in trial directories
5. Model selection will automatically use this data when available


In [165]:
from selection.trial_finder import find_best_trials_for_backbones
from common.shared.platform_detection import detect_platform
import json
from pathlib import Path

def get_trial_hash_info(trial_dir):
    """Extract study_key_hash, trial_key_hash, and trial_number from trial_meta.json if available."""
    trial_meta_path = Path(trial_dir) / "trial_meta.json"
    if not trial_meta_path.exists():
        return None, None, None
    try:
        with open(trial_meta_path, "r") as f:
            meta = json.load(f)
        return meta.get("study_key_hash"), meta.get("trial_key_hash"), meta.get("trial_number")
    except Exception:
        return None, None, None

# Find best trials for all backbones
environment = detect_platform()
best_trials = find_best_trials_for_backbones(
    backbone_values=backbone_values,
    hpo_studies=hpo_studies if "hpo_studies" in locals() else None,
    hpo_config=hpo_config,
    data_config=data_config,
    root_dir=ROOT_DIR,
    environment=environment,
)

# Print summary with hash-based identifiers
print(f"\n📊 Found {len(best_trials)} best trial(s) from {len(backbone_values)} backbone(s)\n")
for backbone, trial_info in best_trials.items():
    study_name = trial_info.get('study_name', 'unknown')
    
    # Get hash info and trial number from trial directory
    trial_dir = trial_info.get('trial_dir')
    study_key_hash, trial_key_hash, trial_number = None, None, None
    if trial_dir:
        study_key_hash, trial_key_hash, trial_number = get_trial_hash_info(trial_dir)
    
    # Extract trial number from trial_name if not in metadata (fallback)
    if trial_number is None:
        trial_name = trial_info.get('trial_name', '')
        if trial_name:
            import re
            match = re.match(r"trial_(\d+)_", trial_name)
            if match:
                trial_number = int(match.group(1))
    
    # Format output with hash-based identifiers (matching MLflow UI naming)
    if study_key_hash and trial_key_hash and trial_number is not None:
        identifier = f"study-{study_key_hash[:8]}, trial-{trial_key_hash[:8]}, t{trial_number}"
    elif study_key_hash and trial_key_hash:
        identifier = f"study-{study_key_hash[:8]}, trial-{trial_key_hash[:8]}"
    elif trial_number is not None:
        identifier = f"t{trial_number}"
    else:
        identifier = "unknown"
    
    accuracy = trial_info.get('accuracy', 0)
    print(f"  ✓ {backbone}: {identifier} ({study_name})")
    print(f"    Best {hpo_config['objective']['metric']}: {accuracy:.4f}")


2026-01-11 23:05:13,681 - evaluation.selection.trial_finder - INFO - Looking for best trial for distilbert (distilbert)...
2026-01-11 23:05:13,690 - evaluation.selection.trial_finder - INFO - distilbert: Best trial is study-584922ce, trial-c4b9e536, t0 (macro-f1=0.3627)
2026-01-11 23:05:13,691 - evaluation.selection.trial_finder - INFO - Looking for best trial for distilroberta (distilroberta)...
2026-01-11 23:05:13,697 - evaluation.selection.trial_finder - INFO - distilroberta: Best trial is study-25cad2a2, trial-96bf728b, t0 (macro-f1=0.4196)
2026-01-11 23:05:13,698 - evaluation.selection.trial_finder - INFO - Summary: Found 2 / 2 best trials



📊 Found 2 best trial(s) from 2 backbone(s)

  ✓ distilbert: study-584922ce, trial-c4b9e536, t0 (study-584922ce)
    Best macro-f1: 0.3627
  ✓ distilroberta: study-25cad2a2, trial-96bf728b, t0 (study-25cad2a2)
    Best macro-f1: 0.4196


In [166]:
# Run benchmarking on best trials
# Define test_data_path directly
test_data_path = DATASET_LOCAL_PATH / "test.json"  # Use test.json as test_data_path

# Load benchmark config (if available)
benchmark_config = configs.get("benchmark", {})
benchmark_settings = benchmark_config.get("benchmarking", {})

# Get benchmark parameters from config or use defaults
benchmark_batch_sizes = benchmark_settings.get("batch_sizes", [1, 8, 16])
benchmark_iterations = benchmark_settings.get("iterations", 100)
benchmark_warmup = benchmark_settings.get("warmup_iterations", 10)
benchmark_max_length = benchmark_settings.get("max_length", 512)
benchmark_device = benchmark_settings.get("device")

# Create MLflow tracker for benchmarking
    # Note: These imports still work via orchestration facade for backward compatibility
from orchestration.jobs.tracking.mlflow_tracker import MLflowBenchmarkTracker
# Use benchmark experiment name (typically same as HPO experiment with -benchmark suffix)
benchmark_experiment_name = f"{experiment_config.name}-benchmark" if 'experiment_config' in locals() else "resume_ner_baseline-benchmark"
benchmark_tracker = MLflowBenchmarkTracker(benchmark_experiment_name)


if test_data_path and test_data_path.exists():
    benchmark_results = {}
from benchmarking import benchmark_best_trials
from common.shared.platform_detection import detect_platform

# Get environment if not already set
if 'environment' not in locals():
    environment = detect_platform()

# Run benchmarking using orchestrator
benchmark_results = benchmark_best_trials(
    best_trials=best_trials,
    test_data_path=test_data_path,
    root_dir=ROOT_DIR,
    environment=environment,
    data_config=data_config,
    hpo_config=hpo_config,
    benchmark_config=benchmark_config,
    benchmark_batch_sizes=benchmark_batch_sizes,
    benchmark_iterations=benchmark_iterations,
    benchmark_warmup=benchmark_warmup,
    benchmark_max_length=benchmark_max_length,
    benchmark_device=benchmark_device,
    benchmark_tracker=benchmark_tracker,
    backup_enabled=BACKUP_ENABLED,
    backup_to_drive=backup_to_drive if "backup_to_drive" in locals() else None,
    ensure_restored_from_drive=ensure_restored_from_drive if "ensure_restored_from_drive" in locals() else None,
)



2026-01-11 23:05:13,771 - evaluation.benchmarking.orchestrator - INFO - Found refit checkpoint: /workspaces/resume-ner-azureml/outputs/hpo/local/distilbert/study-584922ce/trial-c4b9e536/refit/checkpoint
2026-01-11 23:05:13,775 - evaluation.benchmarking.orchestrator - INFO - Benchmarking distilbert (trial-c4b9e536)...
2026-01-11 23:05:13,891 - evaluation.benchmarking.orchestrator - INFO - [BENCHMARK] Found trial run ID from MLflow: b15e56b3-843... (via trial_key_hash=c4b9e536d3adda29...)
2026-01-11 23:05:13,980 - evaluation.benchmarking.orchestrator - INFO - [BENCHMARK] Found refit run ID from MLflow: 54f58037-4f4...
2026-01-11 23:05:13,980 - evaluation.benchmarking.orchestrator - INFO - [BENCHMARK] Final run IDs: trial=b15e56b3-843..., refit=54f58037-4f4..., sweep=None...
2026-01-11 23:05:13,981 - evaluation.benchmarking.utils - INFO - Running benchmark script: /opt/conda/envs/resume-ner-training/bin/python -u /workspaces/resume-ner-azureml/src/evaluation/benchmarking/cli.py --checkpoi

Loaded 2 test texts
Starting benchmark for checkpoint: /workspaces/resume-ner-azureml/outputs/hpo/local/distilbert/study-584922ce/trial-c4b9e536/refit/checkpoint
Loading tokenizer from /workspaces/resume-ner-azureml/outputs/hpo/local/distilbert/study-584922ce/trial-c4b9e536/refit/checkpoint...
Tokenizer loaded.
Loading model from /workspaces/resume-ner-azureml/outputs/hpo/local/distilbert/study-584922ce/trial-c4b9e536/refit/checkpoint...
Moving model to cpu...
Model loaded and set to eval mode.
Model ready on device: cpu

Benchmarking batch size 1...
  Running 10 warmup iterations, then 10 measurement iterations...
    Warmup: 10 iterations... 10/10 done.
    Measurement: 10 iterations... 10/10 done.
  Mean latency: 163.35 ms
  P95 latency: 179.90 ms
  Throughput: 6.12 docs/sec

Saving results to /workspaces/resume-ner-azureml/outputs/benchmarking/local/distilbert/study-584922ce/trial-c4b9e536/bench-7d5a2dd2/benchmark.json...
Benchmark results saved to /workspaces/resume-ner-azureml/ou

2026-01-11 23:05:25,456 - evaluation.benchmarking.utils - INFO - [Benchmark Run Name] Building run name: trial_id=trial-c4b9e536, root_dir=/workspaces/resume-ner-azureml, config_dir=/workspaces/resume-ner-azureml/config
2026-01-11 23:05:25,457 - infrastructure.naming.mlflow.config - INFO - [Auto-Increment Config] Loading from config_dir=/workspaces/resume-ner-azureml/config, raw_auto_inc_config={'enabled': True, 'processes': {'hpo': True, 'benchmarking': True}, 'format': '{base}.{version}'}
2026-01-11 23:05:25,458 - infrastructure.naming.mlflow.config - INFO - [Auto-Increment Config] Validated config: {'enabled': True, 'processes': {'hpo': True, 'benchmarking': True}, 'format': '{base}.{version}'}, process_type=benchmarking
2026-01-11 23:05:25,459 - orchestration.jobs.tracking.index.version_counter - INFO - [Reserve Version] Starting reservation: counter_key=resume-ner:benchmarking:07e5d4553bbcb352726cc12d1fd9a9433a14..., root_dir=/workspaces/resume-ner-azureml, config_dir=/workspaces/

🏃 View run local_distilbert_benchmark_study-584922ce_trial-c4b9e536_bench-7d5a2dd2_1 at: https://germanywestcentral.api.azureml.ms/mlflow/v2.0/subscriptions/50c06ef8-627b-46d5-b779-d07c9b398f75/resourceGroups/resume_ner_2026-01-02-16-47-05/providers/Microsoft.MachineLearningServices/workspaces/resume-ner-ws/#/experiments/29716cbc-2f1e-485a-87be-3ef5c2f931dd/runs/da2704e1-d1b9-46ba-bcde-891a22728c67
🧪 View experiment at: https://germanywestcentral.api.azureml.ms/mlflow/v2.0/subscriptions/50c06ef8-627b-46d5-b779-d07c9b398f75/resourceGroups/resume_ner_2026-01-02-16-47-05/providers/Microsoft.MachineLearningServices/workspaces/resume-ner-ws/#/experiments/29716cbc-2f1e-485a-87be-3ef5c2f931dd


2026-01-11 23:05:29,445 - evaluation.benchmarking.orchestrator - INFO - Benchmark completed: /workspaces/resume-ner-azureml/outputs/benchmarking/local/distilbert/study-584922ce/trial-c4b9e536/bench-7d5a2dd2/benchmark.json
2026-01-11 23:05:29,446 - evaluation.benchmarking.orchestrator - INFO - Found refit checkpoint: /workspaces/resume-ner-azureml/outputs/hpo/local/distilroberta/study-25cad2a2/trial-96bf728b/refit/checkpoint
2026-01-11 23:05:29,448 - evaluation.benchmarking.orchestrator - INFO - Benchmarking distilroberta (trial-96bf728b)...
2026-01-11 23:05:29,519 - evaluation.benchmarking.orchestrator - INFO - [BENCHMARK] Found trial run ID from MLflow: 8f7f4d6b-075... (via trial_key_hash=96bf728ba9b34f84...)
2026-01-11 23:05:29,692 - evaluation.benchmarking.orchestrator - INFO - [BENCHMARK] Found refit run ID from MLflow: 4452788f-ce3...
2026-01-11 23:05:29,692 - evaluation.benchmarking.orchestrator - INFO - [BENCHMARK] Final run IDs: trial=8f7f4d6b-075..., refit=4452788f-ce3..., swe

Loaded 2 test texts
Starting benchmark for checkpoint: /workspaces/resume-ner-azureml/outputs/hpo/local/distilroberta/study-25cad2a2/trial-96bf728b/refit/checkpoint
Loading tokenizer from /workspaces/resume-ner-azureml/outputs/hpo/local/distilroberta/study-25cad2a2/trial-96bf728b/refit/checkpoint...
Tokenizer loaded.
Loading model from /workspaces/resume-ner-azureml/outputs/hpo/local/distilroberta/study-25cad2a2/trial-96bf728b/refit/checkpoint...
Moving model to cpu...
Model loaded and set to eval mode.
Model ready on device: cpu

Benchmarking batch size 1...
  Running 10 warmup iterations, then 10 measurement iterations...
    Warmup: 10 iterations... 10/10 done.
    Measurement: 10 iterations... 10/10 done.
  Mean latency: 215.16 ms
  P95 latency: 227.06 ms
  Throughput: 4.65 docs/sec

Saving results to /workspaces/resume-ner-azureml/outputs/benchmarking/local/distilroberta/study-25cad2a2/trial-96bf728b/bench-7d5a2dd2/benchmark.json...
Benchmark results saved to /workspaces/resume-ne

2026-01-11 23:05:41,132 - evaluation.benchmarking.utils - INFO - [Benchmark Run Name] Building run name: trial_id=trial-96bf728b, root_dir=/workspaces/resume-ner-azureml, config_dir=/workspaces/resume-ner-azureml/config
2026-01-11 23:05:41,133 - infrastructure.naming.mlflow.config - INFO - [Auto-Increment Config] Loading from config_dir=/workspaces/resume-ner-azureml/config, raw_auto_inc_config={'enabled': True, 'processes': {'hpo': True, 'benchmarking': True}, 'format': '{base}.{version}'}
2026-01-11 23:05:41,134 - infrastructure.naming.mlflow.config - INFO - [Auto-Increment Config] Validated config: {'enabled': True, 'processes': {'hpo': True, 'benchmarking': True}, 'format': '{base}.{version}'}, process_type=benchmarking
2026-01-11 23:05:41,135 - orchestration.jobs.tracking.index.version_counter - INFO - [Reserve Version] Starting reservation: counter_key=resume-ner:benchmarking:94788fb9e5142ac7c03ad8f7ae8fd49c6e96..., root_dir=/workspaces/resume-ner-azureml, config_dir=/workspaces/

🏃 View run local_distilroberta_benchmark_study-25cad2a2_trial-96bf728b_bench-7d5a2dd2_1 at: https://germanywestcentral.api.azureml.ms/mlflow/v2.0/subscriptions/50c06ef8-627b-46d5-b779-d07c9b398f75/resourceGroups/resume_ner_2026-01-02-16-47-05/providers/Microsoft.MachineLearningServices/workspaces/resume-ner-ws/#/experiments/29716cbc-2f1e-485a-87be-3ef5c2f931dd/runs/e3ead64f-7861-4377-8497-5aeda0621bd9
🧪 View experiment at: https://germanywestcentral.api.azureml.ms/mlflow/v2.0/subscriptions/50c06ef8-627b-46d5-b779-d07c9b398f75/resourceGroups/resume_ner_2026-01-02-16-47-05/providers/Microsoft.MachineLearningServices/workspaces/resume-ner-ws/#/experiments/29716cbc-2f1e-485a-87be-3ef5c2f931dd


2026-01-11 23:05:45,242 - evaluation.benchmarking.orchestrator - INFO - Benchmark completed: /workspaces/resume-ner-azureml/outputs/benchmarking/local/distilroberta/study-25cad2a2/trial-96bf728b/bench-7d5a2dd2/benchmark.json
2026-01-11 23:05:45,243 - evaluation.benchmarking.orchestrator - INFO - Benchmarking complete. 2/2 trials benchmarked.
